# Two ways to access data from an ODBC data source

This will show examples using the **odbc + DBI** packages and additional examples using the older **RODBC** package.  

**1) odbc + DBI**: How to Use the New R ODBC Package to Connect to an Oracle Database  
https://blog.redpillanalytics.com/how-to-use-the-new-r-odbc-package-to-connect-to-an-oracle-database-70ce6783e397  

**odbc**: Connect to ODBC Compatible Databases (using the DBI Interface)  
https://cran.r-project.org/web/packages/odbc/index.html  
**DBI**: R Database Interface  
https://cran.r-project.org/web/packages/DBI/index.html  

**2) RODBC**: ODBC Database Access  
https://cran.r-project.org/web/packages/RODBC/index.html  


In [1]:
version

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          4                           
minor          1.3                         
year           2022                        
month          03                          
day            10                          
svn rev        81868                       
language       R                           
version.string R version 4.1.3 (2022-03-10)
nickname       One Push-Up                 

In [2]:
# NOTE: When installing Anaconda3 for Windows, choose the option "Install for: Just Me".  
#   This will install Anaconda under the Users folder which will NOT result any subfolders being write-protected.
#   If Anaconda is installed for All Users, it will be installed under C:\ProgramData\Anaconda3\ and will be write-protected.

install.packages('odbc')  

package 'odbc' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\etorp\AppData\Local\Temp\Rtmp0gXBoC\downloaded_packages


In [3]:
# DBI is useful for creating R code to query an ODBC database without relying on the DSN setting (may be unique to user)
install.packages("DBI")

package 'DBI' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\etorp\AppData\Local\Temp\Rtmp0gXBoC\downloaded_packages


### Querring data from MS SQL Server

Setting up an ODBC connection with MS SQL Server on Windows  
https://www.r-bloggers.com/2018/05/setting-up-an-odbc-connection-with-ms-sql-server-on-windows/  
NOTE: Using DBI allows users to access an ODBC data source WITHOUT relying on the DSN that may be unique to the user's computer.  

The Connection Strings Reference  
https://www.connectionstrings.com/  
   Server=myServerAddress;Database=myDataBase;User Id=myUsername;Password=myPassword;  

===============  
R Square Academy: R, Databases & SQL  
https://rdbsql.rsquaredacademy.com/dbi.html  

  
Database Queries With R  
https://solutions.posit.co/connections/db/getting-started/database-queries/  

You can query your data with DBI by using the dbGetQuery() function. Simply paste your SQL code into the R function as a quoted string. This method is sometimes referred to as pass through SQL code, and is probably the simplest way to query your data. Care should be used to escape your quotes as needed. For example, 'yes' is written as \'yes\'.  

dbGetQuery(con,'  
  select "month_idx", "year", "month",  
  sum(case when "term_deposit" = \'yes\' then 1.0 else 0.0 end) as subscribe,  
  count(*) as total  
  from "bank"  
  group by "month_idx", "year", "month"  
')  





In [4]:
library(odbc)
library(DBI)

# Connect without a DSN (using DBI)
db <- DBI::dbConnect(odbc::odbc(),
                     Driver = 'ODBC Driver 17 for SQL Server',
                     Server = 'localhost\\SQLEXPRESS',
                     Database = "AdventureWorksLT2019",
                     trusted_connection = 'yes',
                     Port = 1433
                     )
db

<OdbcConnection> dbo@ThinkPad-X1\SQLEXPRESS
  Database: AdventureWorksLT2019
  Microsoft SQL Server Version: 16.00.1050

In [18]:
# QUESTION: Is it preferable to use "con" instead of "db"?

# dbListTables(db)   # lists all the tables in the database

dbListTables(db, table_name = "P%")   # List tables beginning with P


# dbReadTable() will read a full table into an R data.frame().
# data <- dbReadTable(con, "flights")
data <- dbReadTable(db, 'Product')


# data <- dbReadTable(db, "Product")



[1] "Product"                               
 [2] "ProductCategory"                       
 [3] "ProductDescription"                    
 [4] "ProductModel"                          
 [5] "ProductModelProductDescription"        
 [6] "PARAMETERS"                            
 [7] "parameter_type_usages"                 
 [8] "parameter_xml_schema_collection_usages"
 [9] "parameters"                            
[10] "partition_functions"                   
[11] "partition_parameters"                  
[12] "partition_range_values"                
[13] "partition_schemes"                     
[14] "partitions"                            
[15] "periods"                               
[16] "plan_guides"                           
[17] "procedures"

ERROR: Error: nanodbc/nanodbc.cpp:1752: 42000: [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'Product'.  [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Statement(s) could not be prepared. 
<SQL> 'SELECT * FROM "Product"'
